<a href="https://colab.research.google.com/github/mapcrafter2048/Literature-Review-Generator-ML-17/blob/main/t5_pytorch_sliding_window.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.10


Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
  Using cached tensorflow-2.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
  Using cached tensorboard-2.10.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl.metadata (1.1 kB)
Using cached tensorflow-2.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.0 MB)
Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
Using cached tensorflow_esti

In [3]:
# !pip install tensorflow==2.10
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install .
!pip install huggingface-hub
!pip install keras_nlp
!pip install datasets
!pip install nltk
!pip install rouge-score

import os
import logging
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datasets import load_dataset, DatasetDict
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AutoTokenizer, pipeline

tf.get_logger().setLevel(logging.ERROR)

fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.44.0.dev0-py3-none-any.whl size=9404289 sha256=df0c8be2833abf391cd05cf09afba16a02b8d2f93933ff2c6cf49c4ba6540f7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-aagdpb5g/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.2
    Uninstalling transformers-4.43.2:
      Successfully uninstalled transformers-4.43.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23

In [6]:
!pip install torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 88.8 MB/s eta 0:00:00


In [7]:
import logging
import numpy as np
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset

# Set logging level
logging.basicConfig(level=logging.ERROR)

# Constants
DATASET_SIZE = 2400000
SAMPLE_FRACTION = 0.004
TRAIN_TEST_SPLIT = 0.1
MAX_INPUT_LENGTH = 512  # Use 512 for T5
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
MAX_EPOCHS = 2
MODEL_CHECKPOINT = "t5-small"

In [8]:
# Load dataset
data = load_dataset("scillm/scientific_papers-archive")

In [9]:


# Load dataset
# data = load_dataset("scillm/scientific_papers-archive")

# Downsample the dataset to 0.4%
sample_size = int(DATASET_SIZE * SAMPLE_FRACTION)
data = data.shuffle(seed=42)
sampled_data = data['train'].train_test_split(train_size=sample_size, seed=42)['train']

# Split the dataset into train and test
train_test_data = sampled_data.train_test_split(test_size=TRAIN_TEST_SPLIT, seed=42)
train_data = train_test_data['train']
test_data = train_test_data['test']

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Preprocess function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

train_dataset = tokenized_train.with_format("torch").to(torch.utils.data.DataLoader(
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
))
test_dataset = tokenized_test.with_format("torch").to(torch.utils.data.DataLoader(
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
))

# Training and evaluation code can be added here


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/8640 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4112: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/960 [00:00<?, ? examples/s]

AttributeError: 'Dataset' object has no attribute 'to'

In [3]:
import logging
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset

# Set logging level
logging.basicConfig(level=logging.ERROR)

# Constants
DATASET_SIZE = 2400000
SAMPLE_FRACTION = 0.004
TRAIN_TEST_SPLIT = 0.1
MAX_INPUT_LENGTH = 512  # Use 512 for T5
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
MAX_EPOCHS = 2
MODEL_CHECKPOINT = "t5-small"

In [5]:
 data = load_dataset("scillm/scientific_papers-archive")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:


# Load dataset
# data = load_dataset("scillm/scientific_papers-archive")

# Downsample the dataset to 0.4%
sample_size = int(DATASET_SIZE * SAMPLE_FRACTION)
data = data.shuffle(seed=42)
sampled_data = data['train'].train_test_split(train_size=sample_size, seed=42)['train']

# Split the dataset into train and test
train_test_data = sampled_data.train_test_split(test_size=TRAIN_TEST_SPLIT, seed=42)
train_data = train_test_data['train']
test_data = train_test_data['test']

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Preprocess function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=MAX_TARGET_LENGTH, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

# Convert to PyTorch DataLoader
def create_dataset(data, batch_size):
    def collate_fn(batch):
        return data_collator([torch.tensor(d) for d in batch])

    return DataLoader(data, batch_size=batch_size, collate_fn=collate_fn)

train_dataset = create_dataset(tokenized_train, BATCH_SIZE)
test_dataset = create_dataset(tokenized_test, BATCH_SIZE)

# Training and evaluation code can be added here


Map:   0%|          | 0/8640 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4112: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/960 [00:00<?, ? examples/s]

In [8]:
from transformers import AdamW
# from datasets import load_dataset
from tqdm import tqdm

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
def create_dataset(data, batch_size):
    def collate_fn(batch):
        # Convert each dictionary in the batch to a list of tensors
        tensors = [{key: torch.tensor(value) for key, value in example.items()} for example in batch]
        return data_collator(tensors)

    return DataLoader(data, batch_size=batch_size, collate_fn=collate_fn)

In [14]:
def create_dataset(data, batch_size):
    def collate_fn(batch):
        # The batch is already a dictionary of lists
        return data_collator(batch)

    return DataLoader(data, batch_size=batch_size, collate_fn=collate_fn)

In [15]:
# Training loop
def train():
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataset, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_dataset)

# Evaluation loop
def evaluate():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(test_dataset, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
    return total_loss / len(test_dataset)

# Training and evaluation
for epoch in range(MAX_EPOCHS):
    print(f"Epoch {epoch + 1}/{MAX_EPOCHS}")
    train_loss = train()
    print(f"Training Loss: {train_loss:.4f}")

    eval_loss = evaluate()
    print(f"Evaluation Loss: {eval_loss:.4f}")

Epoch 1/2


Training:   0%|          | 0/1080 [00:00<?, ?it/s]


RuntimeError: Could not infer dtype of dict

In [1]:
import logging
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, AdamW
from datasets import load_dataset
from tqdm import tqdm

# Set logging level
logging.basicConfig(level=logging.ERROR)

# Constants
DATASET_SIZE = 2400000
SAMPLE_FRACTION = 0.004
TRAIN_TEST_SPLIT = 0.1
MAX_INPUT_LENGTH = 512  # Use 512 for T5
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
MAX_EPOCHS = 2
MODEL_CHECKPOINT = "t5-small"

# Load dataset
data = load_dataset("scillm/scientific_papers-archive")

# Downsample the dataset to 0.4%
sample_size = int(DATASET_SIZE * SAMPLE_FRACTION)
data = data.shuffle(seed=42)
sampled_data = data['train'].train_test_split(train_size=sample_size, seed=42)['train']

# Split the dataset into train and test
train_test_data = sampled_data.train_test_split(test_size=TRAIN_TEST_SPLIT, seed=42)
train_data = train_test_data['train']
test_data = train_test_data['test']

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Preprocess function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=MAX_TARGET_LENGTH, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

# Convert to PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.input_ids = torch.tensor(data["input_ids"])
        self.attention_mask = torch.tensor(data["attention_mask"])
        self.labels = torch.tensor(data["labels"])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

train_dataset = CustomDataset(tokenized_train)
test_dataset = CustomDataset(tokenized_test)

# Convert to DataLoader
def create_dataloader(dataset, batch_size):
    return DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)

train_dataloader = create_dataloader(train_dataset, BATCH_SIZE)
test_dataloader = create_dataloader(test_dataset, BATCH_SIZE)

# Setup for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
def train():
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_dataloader)

# Evaluation loop
def evaluate():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
    return total_loss / len(test_dataloader)

# Training and evaluation
for epoch in range(MAX_EPOCHS):
    print(f"Epoch {epoch + 1}/{MAX_EPOCHS}")
    train_loss = train()
    print(f"Training Loss: {train_loss:.4f}")

    eval_loss = evaluate()
    print(f"Evaluation Loss: {eval_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/960 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4112: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2


Training:   0%|          | 0/1080 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Training: 100%|██████████| 1080/1080 [06:01<00:00,  2.99it/s]


...

Evaluation Loss: 2.3611
Epoch 2/2


Training: 100%|██████████| 1080/1080 [06:01<00:00,  2.99it/s]


Training Loss: 2.4513


Evaluating: 100%|██████████| 120/120 [00:13<00:00,  8.64it/s]

Evaluation Loss: 2.2732


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Constants
MODEL_CHECKPOINT = "t5-small"
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Ensure model is in evaluation mode and on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Generating summaries
def generate_summaries(input_texts):
    # Tokenize the input texts
    inputs = tokenizer(input_texts, return_tensors="pt", max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length").to(device)

    # Generate summaries
    summaries = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=MAX_TARGET_LENGTH,
        num_beams=4,               # Beam search for better summaries
        early_stopping=True
    )

    # Decode the generated summaries
    return tokenizer.batch_decode(summaries, skip_special_tokens=True)

# Example usage
sample_inputs = [
    "Quantum computing is revolutionizing the field of computation by leveraging quantum mechanics to solve problems beyond the reach of classical computers. This paper offers a comprehensive review of recent advancements in quantum computing, focusing on innovations in quantum hardware such as superconducting and topological qubits, and notable quantum algorithms including Shor’s and Grover’s algorithms that provide significant speedups over classical methods. We also examine progress in error correction techniques like surface codes and cat codes, which are crucial for managing quantum decoherence and scaling quantum systems. Additionally, the paper explores the transformative applications of quantum computing in areas such as cryptography, where it poses both challenges and opportunities, material science with its potential for groundbreaking simulations, and artificial intelligence, where quantum algorithms could enhance machine learning and optimization tasks. Despite these advancements, the field faces ongoing challenges, including the need for scalable systems and practical applications, making future research essential for realizing quantum computing’s full potential.",

]
generated_summaries = generate_summaries(sample_inputs)

# Print results
for text, summary in zip(sample_inputs, generated_summaries):
    print(f"Input: {text}")
    print(f"Summary: {summary}\n")


Input: Quantum computing is revolutionizing the field of computation by leveraging quantum mechanics to solve problems beyond the reach of classical computers. This paper offers a comprehensive review of recent advancements in quantum computing, focusing on innovations in quantum hardware such as superconducting and topological qubits, and notable quantum algorithms including Shor’s and Grover’s algorithms that provide significant speedups over classical methods. We also examine progress in error correction techniques like surface codes and cat codes, which are crucial for managing quantum decoherence and scaling quantum systems. Additionally, the paper explores the transformative applications of quantum computing in areas such as cryptography, where it poses both challenges and opportunities, material science with its potential for groundbreaking simulations, and artificial intelligence, where quantum algorithms could enhance machine learning and optimization tasks. Despite these adva